In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

device = torch.device("cuda")
"""
from braindecode.datasets import TUHAbnormal
num_workers = 32
tuh_data_dir = "/home/jovyan/mne_data/TUH_PRE/tuh_eeg_abnormal_clip/v2.0.0/edf/"

dataset = TUHAbnormal(
    path=tuh_data_dir,
    recording_ids=None,#range(n_recordings_to_load),  # loads the n chronologically first recordings
    target_name="report",  # age, gender, pathology
    preload=False,
    add_physician_reports=True,
    n_jobs=num_workers)

# ## Preprocessing

# text preprocessing
from EEGClip.text_preprocessing import text_preprocessing
embs_df = text_preprocessing(dataset.description)
embs_df = embs_df[["report"]]
"""
embs_df = pd.read_csv("/home/jovyan/EEGClip/scripts/text_preprocessing/embs_df.csv")


/home/jovyan/test_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def sentence_embedder(sentence,tokenizer,model):
    
    desc_tokenized = tokenizer(sentence, return_tensors="pt", max_length=512, truncation=True, padding='max_length').to(device)
    outputs = model(**desc_tokenized)
    emb = outputs.to_tuple()[0][0][0].detach().cpu().numpy()
    return emb


model_name = "microsoft/BioGPT-Large-PubMedQA"
"""
bert-base-uncased
"BAAI/bge-large-en-v1.5"
medicalai/ClinicalBERT
hkunlp/instructor-xl
microsoft/BioGPT-Large-PubMedQA
microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224
"""
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

model.to(device)

embs = []

import numpy as np
for i, r in enumerate(embs_df['report']):
    if i % 100:
        print(i,'/',len(embs_df['report']))
    emb = sentence_embedder(r,tokenizer,model)
    
    embs.append(emb)


Some weights of the model checkpoint at microsoft/BioGPT-Large-PubMedQA were not used when initializing BioGptModel: ['output_projection.weight']
- This IS expected if you are initializing BioGptModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BioGptModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OutOfMemoryError: CUDA out of memory. Tried to allocate 40.00 MiB (GPU 0; 10.75 GiB total capacity; 524.96 MiB already allocated; 35.62 MiB free; 530.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
embs_df[model_name] = np.array(embs).tolist()

In [13]:
embs_df.iloc[0]["microsoft/BioGPT-Large-PubMedQA"]

[-0.24498771131038666,
 -1.5417234897613525,
 -0.7811378836631775,
 1.020910620689392,
 1.796195149421692,
 -1.2810695171356201,
 0.10497316718101501,
 -0.22209136188030243,
 -1.6525968313217163,
 0.18278852105140686,
 0.4942399859428406,
 -1.0426793098449707,
 -0.5723406076431274,
 -0.5977254509925842,
 1.0945106744766235,
 -2.8077340126037598,
 1.4699848890304565,
 -1.0584434270858765,
 -1.5596340894699097,
 0.5415316224098206,
 -0.16645972430706024,
 0.9139601588249207,
 0.2923087179660797,
 0.5900189280509949,
 0.6894989609718323,
 0.8915250301361084,
 -0.2708861529827118,
 1.402266263961792,
 0.5554729104042053,
 0.20936229825019836,
 -1.800148367881775,
 0.5759848356246948,
 0.5244067311286926,
 -1.909342646598816,
 -0.5244348049163818,
 -0.6986033320426941,
 1.6588935852050781,
 2.6781551837921143,
 1.386518120765686,
 -0.8828237056732178,
 1.6096152067184448,
 2.053314208984375,
 0.7742587327957153,
 -0.020037522539496422,
 0.6014757752418518,
 -0.3577083647251129,
 -0.13282945

In [ ]:
embs_df.to_csv("/home/jovyan/EEGClip/scripts/text_preprocessing/embs_df.csv")

In [2]:
import copy
import pandas as pd
embs_df = pd.read_csv("/home/jovyan/EEGClip/scripts/text_preprocessing/embs_df.csv")
embs_name = "bert-base-uncased"
for r in range(len(embs_df)):
    re = copy.copy(embs_df[embs_name][r])
    # convert the string to array
    re = re.replace('[', '')
    re = re.replace(']', '')
    re = re.replace(',', '')
    re = re.split()
    print(re)
    re = [float(i) for i in re]
    embs_df[embs_name][r] = re